In [1]:
import datetime
import string
import random
import os
os.chdir("C:\\AnalyticsPy\\")

from analyticspy.tools.data_table import DataTable

In [2]:
table_len = 1000

table = DataTable({
    "Integers": [random.randint(10,99) for _ in range(table_len)],
    "Strings": [''.join(random.choices(string.ascii_uppercase + string.digits, k=5)) for _ in range(table_len)],
    "Booleans": [bool(random.getrandbits(1)) for _ in range(table_len)]
})
table.display()

Index,IntegersDataType:int,StringsDataType:str,BooleansDataType:bool
0,79,X0ZIT,False
1,86,CTDZ5,True
2,68,IQAP7,True
3,43,D1WZK,False
4,54,RRDPP,False
...,...,...,...
995,93,FLSLU,True
996,17,DLS5O,True
997,70,D8HS7,False
998,28,9OO9N,True


In [3]:
table.insert_column("Floats", [random.random() for _ in range(table_len)])

for row in table:
    row["Multiplication"] = row["Integers"] * row["Floats"]

table.insert_column(
    "Dates",
    [datetime.date.today() - datetime.timedelta(days=i) for i in range(table_len)],
    column_index=0)

table.display()

Index,DatesDataType:date,IntegersDataType:int,StringsDataType:str,BooleansDataType:bool,FloatsDataType:float,MultiplicationDataType:float
0,2020-09-26,79,X0ZIT,False,0.10238912196767758,8.088740635446529
1,2020-09-25,86,CTDZ5,True,0.9479238857805974,81.52145417713137
2,2020-09-24,68,IQAP7,True,0.9054264789203592,61.56900056658443
3,2020-09-23,43,D1WZK,False,0.104563314308173,4.496222515251439
4,2020-09-22,54,RRDPP,False,0.21783290727597293,11.762976992902539
...,...,...,...,...,...,...
995,2018-01-05,93,FLSLU,True,0.1440500793651298,13.396657380957073
996,2018-01-04,17,DLS5O,True,0.8127992997407573,13.817588095592873
997,2018-01-03,70,D8HS7,False,0.9110273320740714,63.771913245185
998,2018-01-02,28,9OO9N,True,0.08810802980370314,2.467024834503688


In [4]:
table._data_map

[['Dates', datetime.date],
 ['Integers', int],
 ['Strings', str],
 ['Booleans', bool],
 ['Floats', float],
 ['Multiplication', float]]

In [5]:
table.create_dummies("Booleans", False)
table.display()

Index,DatesDataType:date,IntegersDataType:int,StringsDataType:str,BooleansDataType:bool,BooleansFalseDataType:int,BooleansTrueDataType:int,FloatsDataType:float,MultiplicationDataType:float
0,2020-09-26,79,X0ZIT,False,1,0,0.10238912196767758,8.088740635446529
1,2020-09-25,86,CTDZ5,True,0,1,0.9479238857805974,81.52145417713137
2,2020-09-24,68,IQAP7,True,0,1,0.9054264789203592,61.56900056658443
3,2020-09-23,43,D1WZK,False,1,0,0.104563314308173,4.496222515251439
4,2020-09-22,54,RRDPP,False,1,0,0.21783290727597293,11.762976992902539
...,...,...,...,...,...,...,...,...
995,2018-01-05,93,FLSLU,True,0,1,0.1440500793651298,13.396657380957073
996,2018-01-04,17,DLS5O,True,0,1,0.8127992997407573,13.817588095592873
997,2018-01-03,70,D8HS7,False,1,0,0.9110273320740714,63.771913245185
998,2018-01-02,28,9OO9N,True,0,1,0.08810802980370314,2.467024834503688


In [6]:
print(table)

DataTable(shape=8x1000,bytesize=300401)
Dates: date
Integers: int
Strings: str
Booleans: bool
BooleansFalse: int
BooleansTrue: int
Floats: float
Multiplication: float


In [7]:
import analyticspy.tools.data_flow as atdf
from analyticspy import APP_PATH

atdf.table_to_parquet(table, "TestDataTable", APP_PATH + "\\tests\\")

In [8]:
table = atdf.table_from_parquet("TestDataTable", APP_PATH + "\\tests\\")
table.display()

Index,DatesDataType:date,IntegersDataType:int,StringsDataType:str,BooleansDataType:bool,BooleansFalseDataType:int,BooleansTrueDataType:int,FloatsDataType:float,MultiplicationDataType:float
0,2020-09-26,79,X0ZIT,False,1,0,0.10238912196767758,8.088740635446529
1,2020-09-25,86,CTDZ5,True,0,1,0.9479238857805974,81.52145417713137
2,2020-09-24,68,IQAP7,True,0,1,0.9054264789203592,61.56900056658443
3,2020-09-23,43,D1WZK,False,1,0,0.104563314308173,4.496222515251439
4,2020-09-22,54,RRDPP,False,1,0,0.21783290727597293,11.762976992902539
...,...,...,...,...,...,...,...,...
995,2018-01-05,93,FLSLU,True,0,1,0.1440500793651298,13.396657380957073
996,2018-01-04,17,DLS5O,True,0,1,0.8127992997407573,13.817588095592873
997,2018-01-03,70,D8HS7,False,1,0,0.9110273320740714,63.771913245185
998,2018-01-02,28,9OO9N,True,0,1,0.08810802980370314,2.467024834503688


In [9]:
table_part_one = table.copy
table_part_one.filter(lambda row: row["Integers"] > 50)
table_part_one.display()

Index,DatesDataType:date,IntegersDataType:int,StringsDataType:str,BooleansDataType:bool,BooleansFalseDataType:int,BooleansTrueDataType:int,FloatsDataType:float,MultiplicationDataType:float
0,2020-09-26,79,X0ZIT,False,1,0,0.10238912196767758,8.088740635446529
1,2020-09-25,86,CTDZ5,True,0,1,0.9479238857805974,81.52145417713137
2,2020-09-24,68,IQAP7,True,0,1,0.9054264789203592,61.56900056658443
3,2020-09-22,54,RRDPP,False,1,0,0.21783290727597293,11.762976992902539
4,2020-09-21,99,6LUCK,True,0,1,0.8289826194291213,82.06927932348302
...,...,...,...,...,...,...,...,...
529,2018-01-07,94,WQTNA,True,0,1,0.7111291276118065,66.8461379955098
530,2018-01-06,55,2JT6N,True,0,1,0.08374868426417881,4.606177634529835
531,2018-01-05,93,FLSLU,True,0,1,0.1440500793651298,13.396657380957073
532,2018-01-03,70,D8HS7,False,1,0,0.9110273320740714,63.771913245185


In [10]:
table_part_two = table.copy
table_part_two.filter(lambda row: row["Integers"] <= 50)
table_part_two.display()

Index,DatesDataType:date,IntegersDataType:int,StringsDataType:str,BooleansDataType:bool,BooleansFalseDataType:int,BooleansTrueDataType:int,FloatsDataType:float,MultiplicationDataType:float
0,2020-09-23,43,D1WZK,False,1,0,0.104563314308173,4.496222515251439
1,2020-09-20,37,L2JA9,False,1,0,0.8531559530323103,31.56677026219548
2,2020-09-17,43,WFLHD,False,1,0,0.5382981104911942,23.14681875112135
3,2020-09-16,30,6T748,False,1,0,0.5364774173593463,16.094322520780388
4,2020-09-14,32,1RQI6,False,1,0,0.06749268701963307,2.159765984628258
...,...,...,...,...,...,...,...,...
461,2018-01-12,23,FV7SI,False,1,0,0.8222483906527474,18.91171298501319
462,2018-01-09,16,INJDC,True,0,1,0.2000797619456326,3.2012761911301215
463,2018-01-08,29,SQTQ9,False,1,0,0.7530168788345791,21.837489486202795
464,2018-01-04,17,DLS5O,True,0,1,0.8127992997407573,13.817588095592873


In [11]:
table_part_one.concatenate(table_part_two, ["Dates", "Floats", "Strings"], ["Dates", "Multiplication", "Strings"])
table_part_one.sort(["Dates"])
table_part_one.display()

Index,DatesDataType:date,IntegersDataType:int,StringsDataType:str,BooleansDataType:bool,BooleansFalseDataType:int,BooleansTrueDataType:int,FloatsDataType:float,MultiplicationDataType:float
0,2018-01-01,95,0AWHI,False,1,0,0.5135152044391742,48.78394442172155
1,2018-01-02,None,9OO9N,None,None,None,2.467024834503688,None
2,2018-01-03,70,D8HS7,False,1,0,0.9110273320740714,63.771913245185
3,2018-01-04,None,DLS5O,None,None,None,13.817588095592873,None
4,2018-01-05,93,FLSLU,True,0,1,0.1440500793651298,13.396657380957073
...,...,...,...,...,...,...,...,...
995,2020-09-22,54,RRDPP,False,1,0,0.21783290727597293,11.762976992902539
996,2020-09-23,None,D1WZK,None,None,None,4.496222515251439,None
997,2020-09-24,68,IQAP7,True,0,1,0.9054264789203592,61.56900056658443
998,2020-09-25,86,CTDZ5,True,0,1,0.9479238857805974,81.52145417713137


In [12]:
table_to_join = DataTable({
    "Dates": [datetime.date.today() - datetime.timedelta(days=i) for i in range(table_len//2)],
    "RegistredValues": [round(random.random()*100,4) for _ in range(table_len//2)]
})
table_to_join.display()

Index,DatesDataType:date,RegistredValuesDataType:float
0,2020-09-26,63.2957
1,2020-09-25,89.6182
2,2020-09-24,22.3415
3,2020-09-23,91.6807
4,2020-09-22,85.3242
...,...,...
495,2019-05-20,54.6083
496,2019-05-19,19.0561
497,2019-05-18,58.27
498,2019-05-17,85.105


In [13]:
table.join(table_to_join, "Dates", "Dates")
table.display()

Index,DatesDataType:date,IntegersDataType:int,StringsDataType:str,BooleansDataType:bool,BooleansFalseDataType:int,BooleansTrueDataType:int,FloatsDataType:float,MultiplicationDataType:float,RegistredValuesDataType:float
0,2020-09-26,79,X0ZIT,False,1,0,0.10238912196767758,8.088740635446529,63.2957
1,2020-09-25,86,CTDZ5,True,0,1,0.9479238857805974,81.52145417713137,89.6182
2,2020-09-24,68,IQAP7,True,0,1,0.9054264789203592,61.56900056658443,22.3415
3,2020-09-23,43,D1WZK,False,1,0,0.104563314308173,4.496222515251439,91.6807
4,2020-09-22,54,RRDPP,False,1,0,0.21783290727597293,11.762976992902539,85.3242
...,...,...,...,...,...,...,...,...,...
995,2018-01-05,93,FLSLU,True,0,1,0.1440500793651298,13.396657380957073,None
996,2018-01-04,17,DLS5O,True,0,1,0.8127992997407573,13.817588095592873,None
997,2018-01-03,70,D8HS7,False,1,0,0.9110273320740714,63.771913245185,None
998,2018-01-02,28,9OO9N,True,0,1,0.08810802980370314,2.467024834503688,None


In [14]:
from statistics import mean, stdev

mu = mean(table["Multiplication"])
sigma = stdev(table["Multiplication"])

table.apply("Multiplication", lambda row: round((row-mu)/sigma, 4))

In [15]:
table.display()

Index,DatesDataType:date,IntegersDataType:int,StringsDataType:str,BooleansDataType:bool,BooleansFalseDataType:int,BooleansTrueDataType:int,FloatsDataType:float,MultiplicationDataType:float,RegistredValuesDataType:float
0,2020-09-26,79,X0ZIT,False,1,0,0.10238912196767758,-0.8611,63.2957
1,2020-09-25,86,CTDZ5,True,0,1,0.9479238857805974,2.5421,89.6182
2,2020-09-24,68,IQAP7,True,0,1,0.9054264789203592,1.6174,22.3415
3,2020-09-23,43,D1WZK,False,1,0,0.104563314308173,-1.0276,91.6807
4,2020-09-22,54,RRDPP,False,1,0,0.21783290727597293,-0.6909,85.3242
...,...,...,...,...,...,...,...,...,...
995,2018-01-05,93,FLSLU,True,0,1,0.1440500793651298,-0.6151,None
996,2018-01-04,17,DLS5O,True,0,1,0.8127992997407573,-0.5956,None
997,2018-01-03,70,D8HS7,False,1,0,0.9110273320740714,1.7195,None
998,2018-01-02,28,9OO9N,True,0,1,0.08810802980370314,-1.1217,None
